In [1]:
import os
import sys
import array
import random
import wave

import numpy as np

repos_dir = r'/home/takkan/repos'
#repos_dir = r'/home/akikun/repos'
sys.path.append(repos_dir)

imgan_dir = os.path.join(repos_dir, 'Intelligibility-MetricGAN')
sys.path.append(imgan_dir)

from sak import display as dp
from sak.nele import add_noise

In [2]:
# Signal-to-Noise
snr = -10.0

# open iMetricGAN sample files.
wav_dir = os.path.join(imgan_dir, 'JR_database')
clean_file = os.path.join(wav_dir, 'Train/Clean/Train_001.wav')
noise_file = os.path.join(wav_dir, 'Train/Noise/Train_001.wav')
#wav_dir = os.path.join(imgan_dir, 'database')
#clean_file = os.path.join(wav_dir, 'Train', 'Clean', 'Train_1.wav')
#noise_file = os.path.join(wav_dir, 'Train', 'Noise', 'Train_1.wav')
clean_wav = wave.open(clean_file, "r")
noise_wav = wave.open(noise_file, "r")

In [3]:
'''
Reference: https://github.com/Sato-Kunihiko/audio-SNR
'''
def cal_amp(wf):
    buffer = wf.readframes(wf.getnframes())
    amptitude = (np.frombuffer(buffer, dtype="int16")).astype(np.float64)
    return amptitude

def cal_rms(amp):
    return np.sqrt(np.mean(np.square(amp), axis=-1))

def cal_adjusted_rms(clean_rms, snr):
    a = float(snr) / 20
    noise_rms = clean_rms / (10**a) 
    return noise_rms


In [4]:
# Calculation of amp
clean_amp = cal_amp(clean_wav)
noise_amp = cal_amp(noise_wav)

print(clean_amp, noise_amp)

[0. 0. 0. ... 0. 0. 0.] [-1.000e+00  3.000e+00 -4.000e+00 ... -3.428e+03 -1.854e+03 -2.371e+03]


In [5]:
# Calculation of RMS
start = random.randint(0, len(noise_amp)-len(clean_amp))
clean_rms = cal_rms(clean_amp)
split_noise_amp = noise_amp[start: start + len(clean_amp)]
noise_rms = cal_rms(split_noise_amp)

print(clean_rms, noise_rms)

3887.8149914806863 3280.3736778128223


In [6]:
# Synthesize waveforms of any size
adjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)
        
adjusted_noise_amp = split_noise_amp * (adjusted_noise_rms / noise_rms) 
mixed_amp = (clean_amp + adjusted_noise_amp)

# Normalized so as not to crack the sound
if (mixed_amp.max(axis=0) > 32767): 
    mixed_amp = mixed_amp * (32767/mixed_amp.max(axis=0))
    clean_amp = clean_amp * (32767/mixed_amp.max(axis=0))
    adjusted_noise_amp = adjusted_noise_amp * (32767/mixed_amp.max(axis=0))


In [7]:
# Save the waveform as a wav file
noisy_wave = wave.Wave_write('output_noisy_file.wav')
noisy_wave.setparams(clean_wav.getparams())
noisy_wave.writeframes(array.array('h', mixed_amp.astype(np.int16)).tostring() )
noisy_wave.close()


## listen the audio.

In [8]:
dp.disp_wav('output_noisy_file.wav')

In [9]:
add_noise(clean_file, noise_file, 'output_noisy_file_2.wav', -10.0)

/home/takkan/miniconda3/envs/iMetricGAN/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  """Entry point for launching an IPython kernel.


In [10]:
dp.disp_wav('output_noisy_file_2.wav')